In [64]:
import numpy as np
import itertools

In [65]:
i1 = 0.05
i2 = 0.10

b1 = 0.35

w1 = 0.15
w2 = 0.20
w3 = 0.25
w4 = 0.30

o1 = 0.01
o2 = 0.99

b2 = 0.60

w5 = 0.4
w6 = 0.45
w7 = 0.5
w8 = 0.55

In [66]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [67]:
def calculate_weight_set_dimensions(dimensions):

    a, b = itertools.tee(dimensions[::-1])
    next(b, None)
    weight_set_dimensions = list(zip(a, b))[::-1]
    
    return weight_set_dimensions

In [68]:
dimensions = [2,2,2]

calculate_weight_set_dimensions(dimensions)

[(2, 2), (2, 2)]

In [69]:
def initialise_weights(dimensions):
    
    # For single hidden layer neural network there will be 2 sets of weights;
    # 1- one set to hidden layer
    # 2- one set from hidden layer
    # number of weight sets = no_of_hidden_layers + 1
    
    weight_dims = calculate_weight_set_dimensions(dimensions)
    no_of_weight_sets = len(weight_dims)
    
    # W_set holds weight sets such as w1, w2, w3 etc.
    W = np.empty_like(range(no_of_weight_sets), dtype=object)
    B = np.empty_like(range(no_of_weight_sets), dtype=object)
    for index, (row, column) in enumerate(weight_dims):
        W[index] = np.random.rand(row, column)
        B[index] = np.random.rand()
    return W, B

In [70]:
def forwardpass(X, W, B):
    weight_dims = calculate_weight_set_dimensions(dimensions)
    no_of_weight_sets = len(weight_dims)

    Z = np.empty_like(range(no_of_weight_sets + 1), dtype=object)
    A = np.empty_like(range(no_of_weight_sets + 1), dtype=object)
    A[0] = X
    Z[0] = None
    B[0] = 0.35
    B[1] = 0.6
    for index in range(no_of_weight_sets):
        Z[index + 1] = W[index] @ A[index] + B[index]
        A[index + 1] = sigmoid(Z[index + 1])
    return A, Z

In [71]:
def calculate_error(y, y_hat):
    return 0.5 * np.square(y - y_hat)

In [82]:
def backpropagation(A, Z, Y, W):
    weight_dims = calculate_weight_set_dimensions(dimensions)
    no_of_weight_sets = len(weight_dims)

    delta_final = (A[-1] - Y) * (sigmoid_prime(Z[-1]))
    
    gradient = np.empty_like(range(no_of_weight_sets), dtype=object)
    # delta = dLZ
    delta = np.empty_like(range(no_of_weight_sets), dtype=object)
    delta[-1] = delta_final
    
    # here Z[index+1] is passed instead of Z[index]
    # this is because Z[0] is none. so Z[index+1] is effectively
    # Z[index]
    for index in reversed(range(no_of_weight_sets - 1)): # 1 is substracted as we have already calculated delta_final above
        delta[index] = W[index+1].T @ delta[index+1] * sigmoid_prime(Z[index + 1])
    
    # calculate the gradient
    for index in range(no_of_weight_sets):
        gradient[index] = delta[index] @ A[index].T
    
    alfa = 0.5
    for index, _ in enumerate(W):
        W[index] = W[index] - alfa * gradient[index]
    
    return delta, gradient, W

In [83]:
X = np.array([0.05, 0.10]).reshape((dimensions[0], 1))
Y = np.array([0.01, 0.99]).reshape((dimensions[-1], 1))

W, B = initialise_weights(dimensions)

initial_weights = W[0]
initial_weights[0][0] = 0.15
initial_weights[0][1] = 0.2
initial_weights[1][0] = 0.25
initial_weights[1][1] = 0.30
B[0] = 0.35

second_set_of_weights = W[1]
second_set_of_weights[0][0] = 0.4
second_set_of_weights[0][1] = 0.45
second_set_of_weights[1][0] = 0.5
second_set_of_weights[1][1] = 0.55
B[1] = 0.6

A, Z = forwardpass(X, W, B)
J = calculate_error(Y, A[-1])

In [84]:
delta, gradient, W = backpropagation(A, Z, Y, W)

In [85]:
gradient[0]

array([[0.00043857, 0.00087714],
       [0.00049771, 0.00099543]])

In [86]:
gradient[1]

array([[ 0.08216704,  0.08266763],
       [-0.02260254, -0.02274024]])

In [87]:
W[0]

array([[0.14978072, 0.19956143],
       [0.24975114, 0.29950229]])

In [88]:
W[0]

array([[0.14978072, 0.19956143],
       [0.24975114, 0.29950229]])

In [89]:
W[1]

array([[0.35891648, 0.40866619],
       [0.51130127, 0.56137012]])

In [ ]:
J

In [ ]:
dLZ_final = (A[-1] - Y) * (sigmoid_prime(Z[-1]))

In [ ]:
dLZ_final

In [ ]:
A[1]

In [ ]:
weight_set = dLZ_final @ A[1].T
weight_set

In [ ]:
no_of_weight_sets = 2

In [ ]:
dLZ = np.empty_like(range(no_of_weight_sets), dtype=object)
dLZ[-1] = dLZ_final

In [ ]:
dLZ[1]

In [ ]:
for index in reversed(range(no_of_weight_sets - 1)):
    print(index)
    dLZ[index] = W[index+1].T @ dLZ[index+1] * sigmoid_prime(Z[index + 1])

In [ ]:
dLZ[0] = W[1].T @ dLZ[1] * sigmoid_prime(Z[1])

In [ ]:
dLZ[0] @ X.T

In [ ]:
for element in dLZ[0] @ X.T:
    print(element)

In [ ]:
X